In [4]:
from datetime import datetime
import matplotlib.pyplot as plt
from herbie import Herbie
import numpy as np
import xarray as xr
from matplotlib import rcParams
import geopandas as gpd
from matplotlib import patheffects as path_effects

# Set style for light background with enhanced readability for poster viewing
plt.style.use('default')
plt.rcParams.update({
    'font.family': 'Helvetica',
    'font.size': 24,
    'axes.labelsize': 32,
    'axes.titlesize': 40,
    'xtick.labelsize': 24,
    'ytick.labelsize': 24,
    'legend.fontsize': 24,
    'figure.titlesize': 48
})

def get_gefs_clouds(date):
    H = Herbie(
        date,
        model="gefs",
        product="atmos.5b",
        member="c00"
    )

    try:
        H.download("TCDC")
        ds = H.xarray("TCDC")
        ds.coords['longitude'] = (ds.coords['longitude'] + 180) % 360 - 180
        ds = ds.sel(
            longitude=slice(-111.0, -108.5),  # Extend west to include Wasatch influence
            latitude=slice(41.0, 39.0)        # Include Uinta Mountains to Book Cliffs
        )
        return ds
    except Exception as e:
        print(f"Error processing {date}: {str(e)}")
        return None

def plot_period(dates, ax, title, counties_gdf):
    ax.set_aspect('equal')
    datasets = []

    for date in dates:
        ds = get_gefs_clouds(date)
        if ds is not None:
            datasets.append(ds)

    if datasets:
        composite = xr.concat(datasets, dim='time').mean(dim='time')

        # Create the colorbar label beforehand
        colorbar_label = 'Cloud Cover (%) - Basin Center: 40°13\'18"N, 109°32\'19"W'

        plot = composite.tcc.plot(
            ax=ax,
            cmap='Blues',
            vmin=0,
            vmax=100,
            cbar_kwargs={
                'label': colorbar_label,
                'orientation': 'horizontal',
                'pad': 0.1,
                'shrink': 0.8,
                'aspect': 40,
                'format': lambda x, _: f'{int(x)}',
                'ticks': [0, 20, 40, 60, 80, 100]
            },
            robust=True,
            zorder=1
        )

        # Adjust colorbar label font size after plotting
        cbar = plot.colorbar
        cbar.ax.tick_params(labelsize=24)  # Match the font size for ticks
        # Set the label directly with the desired size
        cbar.set_label(colorbar_label, size=28)

        # Plot county boundaries if available
        if counties_gdf is not None:
            try:
                # Filter for Duchesne and Uintah counties, excluding their shared boundary
                mask = (
                    (counties_gdf['RightCount'].isin(['UINTAH', 'DUCHESNE']) |
                     counties_gdf['LeftCounty'].isin(['UINTAH', 'DUCHESNE'])) &
                    ~((counties_gdf['RightCount'] == 'UINTAH') &
                      (counties_gdf['LeftCounty'] == 'DUCHESNE'))
                )
                counties_filtered = counties_gdf[mask]
                counties_filtered.plot(ax=ax, facecolor='none', edgecolor='black',
                                    linewidth=2.5, alpha=1.0, zorder=5)
                print("Successfully plotted county boundaries")
            except Exception as e:
                print(f"Error plotting county boundaries: {str(e)}")

        ax.set_title(title, pad=30)
        ax.set_xlabel('Longitude (°W)', labelpad=15)
        ax.set_ylabel('Latitude (°N)', labelpad=15)

        ax.set_xticks([-111, -110.5, -110, -109.5, -109, -108.5])
        ax.set_xticklabels(['111°W', '110.5°W', '110°W', '109.5°W', '109°W', '108.5°W'])

        ax.grid(False)

        # Cities with coordinates
        cities = {
            'Duchesne': {'lon': -110.533742, 'lat': 40.193815},
            'Roosevelt': {'lon': -109.989831, 'lat': 40.299167},
            'Vernal': {'lon': -109.528, 'lat': 40.456}
        }

        # Plot cities with larger markers and text
        for city, coords in cities.items():
            ax.plot(coords['lon'], coords['lat'], 'o',
                   color='white', markeredgecolor='black',
                   markeredgewidth=1.5, markersize=12, zorder=10)

            text = ax.text(coords['lon'] + 0.03, coords['lat'] + 0.03,
                         city, fontsize=20, zorder=12)
            text.set_path_effects([path_effects.withStroke(linewidth=3, foreground='white')])

        for spine in ax.spines.values():
            spine.set_linewidth(2.5)
            spine.set_color('black')

# Load the Counties shapefile
try:
    counties_gdf = gpd.read_file('/Users/a02428741/PycharmProjects/snowshadow/notebooks/shps/Counties_LabelLines/Counties_LabelLines.shp')
    # Ensure the CRS is in the correct projection (WGS 84)
    if counties_gdf.crs is not None and counties_gdf.crs != 'EPSG:4326':
        counties_gdf = counties_gdf.to_crs('EPSG:4326')
    print("Successfully loaded and processed shapefile")
except Exception as e:
    print(f"Error loading shapefile: {str(e)}")
    counties_gdf = None

dates_2023 = [
    '2023-02-20 00:00', '2023-02-21 00:00', '2023-02-22 00:00',
    '2023-02-23 00:00', '2023-02-24 00:00', '2023-02-25 00:00',
    '2023-02-26 00:00', '2023-02-27 00:00'
]

dates_2024 = [
    '2024-02-27 00:00', '2024-02-28 00:00', '2024-02-29 00:00',
    '2024-03-01 00:00', '2024-03-02 00:00', '2024-03-03 00:00',
    '2024-03-04 00:00', '2024-03-05 00:00'
]

# Increased figure size for poster
fig = plt.figure(figsize=(36, 24), facecolor='white', dpi=300)
gs = plt.GridSpec(1, 2, wspace=0.2)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])

print("\nProcessing 2023 period...")
plot_period(dates_2023, ax1, 'Case Study 1: February 2023', counties_gdf)

print("\nProcessing 2024 period...")
plot_period(dates_2024, ax2, 'Case Study 2: February/March 2024', counties_gdf)

fig.suptitle('GEFS Analysis: Cloud Cover Distribution based on Case Studies',
             y=1.02, x=0.5, horizontalalignment='center')

# Updated notes with dots and data source information
notes_text = (
    '• GEFS analysis showing mean cloud cover distribution\n'
    '• Uinta Basin region (39.5°N-41°N, 111°W-109°W)\n'
    '• Data retrieved from GEFS through the Herbie data API'
)

plt.figtext(0.02, 0.08, notes_text,
            fontsize=28, ha='left', color='#333333')

plt.subplots_adjust(top=0.85, bottom=0.2, left=0.08, right=0.92)

plt.show()

Successfully loaded and processed shapefile

Processing 2023 period...
✅ Found ┊ model=gefs ┊ product=atmos.5b ┊ 2023-Feb-20 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/opt/anaconda3/lib/python3.12/site-packages/herbie/core.py:1112: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")
/opt/anaconda3/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Found ┊ model=gefs ┊ product=atmos.5b ┊ 2023-Feb-21 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/opt/anaconda3/lib/python3.12/site-packages/herbie/core.py:1112: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")
/opt/anaconda3/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Found ┊ model=gefs ┊ product=atmos.5b ┊ 2023-Feb-22 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


...

✅ Found ┊ model=gefs ┊ product=atmos.5b ┊ 2024-Mar-04 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/opt/anaconda3/lib/python3.12/site-packages/herbie/core.py:1112: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")
/opt/anaconda3/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Found ┊ model=gefs ┊ product=atmos.5b ┊ 2024-Mar-05 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/opt/anaconda3/lib/python3.12/site-packages/herbie/core.py:1112: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")
/opt/anaconda3/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


Successfully plotted county boundaries
